<a href="https://colab.research.google.com/github/ThomasMiconi/MetaMetaLearning/blob/main/MML_DMS_BS500_400T_Simplify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Feb  8 19:55:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import pdb

import scipy
from scipy import ndimage
from scipy import linalg

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import time

import numpy as np
from numpy import fft 

from scipy import io as spio
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.set_printoptions(precision=5) 
np.set_printoptions(precision=5) 



# Specify the test task (and its logical negation, which is also withheld from the training set)
# TESTTASK = 'nand'; TESTTASKNEG = 'and'
TESTTASK = 'dms'; TESTTASKNEG = 'dnms'


LR =  .3 * 1e-2  # Adam LR. 

NBRUNS = 1 # 5          # Number of runs
NBGEN = 1700 # 500      # Number of generations per run


N = 70  # Number of neurons in the RNN.


BS = 500 # 1000         # Batch size, i.e. population size for the evolutionary algorithm. 
assert BS % 2 == 0      # Should be even because of antithetic sampling.

# Same parameters as GR Yang:
TAU =  100  # Neuron membrane constant, in ms
DT = 20     # Duration of a timestep, in ms


# All the following times are in *timesteps*, not ms
T =  50      # Number of *timesteps* per trial
STIMTIME = 20       # Duration of stimulus input, total, *in timesteps* (not ms)
REWARDTIME = 10     # Duration of reward signal period
RESPONSETIME = 10   # Duration of responze period  
STARTRESPONSETIME = 25  # Timestep  at which response period starts
ENDRESPONSETIME = STARTRESPONSETIME + RESPONSETIME
STARTREWARDTIME = 36    # Timsestep at which reward is deliverd and reward signal starts
ENDREWARDTIME = STARTREWARDTIME + REWARDTIME
assert ENDREWARDTIME < T


JINIT = 1.5 #   Scale constant of initial network weights. See Section 2.7 in the MML paper.
TAU_ET = 1000.0    # Time constant of the eligibility trace (in ms)
PROBAMODUL = .1 #       Probability of receiving a random perturbation, for each neuron, at each timestep.
ALPHAMODUL =  .5 #      Scale of the random perturbations
ETA = .03 #             Learning rate for lifetime plasticity
MULOSSTRACE = .9    #   Time constant for the trace of previous losses that serves as a baseline for neuromodulation
MAXDW = 1e-2 #          Maximum delta-weight perimissible for lifetime plasticity (remember that plasticity is only applied once per trial, at reward delivery time)
INITALPHA = .5 #        Initial alpha (plasticity parameter) value


# The name of all the tasks
alltasks = ['01', 'anti01', '10', 'anti10', 'watchstim1', 'watchstim2' ,'dms',  'antiwatchstim2', 'antiwatchstim1', 'or', 'and', 'nor', 'nand', 'dnms']


NBRESPS = 1 # Only 1 possible response ("1") in addition to the null "no-response" ("0"), for now.


NBSTIMNEURONS = 2   # 2 Stimulus neurons. Stimuli are binary, so both neurons receive opposite-valued inputs (or 0)
NBREWARDNEURONS = 2 # 2 reward signal (correct/incorrect) neurons. Same as for stimulus neurons.
NBNOISENEURONS = 1  # Noise neurons (not used in this code)
NBBIASNEURONS = 1   # Bias neurons. Activations clamped to BIASVALUE.
NBINPUTNEURONS = NBSTIMNEURONS + NBREWARDNEURONS +  NBNOISENEURONS + NBBIASNEURONS    # The first NBINPUTS neurons in the network are neurons (includes the bias, noise and reward inputs)
NBRESPNEURONS = 1 + NBRESPS  # 0-resp plus all other resps (for now, only 1)
NBAUXNEURONS = 0 #  No Auxiliary neurons for now.
NBOUTPUTNEURONS = NBRESPNEURONS + NBAUXNEURONS      # The last NBOUTPUTNEURONS neurons in the network are output neurons
NBRESPSIGNALNEURONS = NBRESPNEURONS     # Neurons that receive the response signal ("what response did I just give?")
STIMNEURONS = np.arange(NBSTIMNEURONS)
INPUTNEURONS = np.arange(NBINPUTNEURONS)
OUTPUTNEURONS = np.arange(N-NBOUTPUTNEURONS, N)
RESPNEURONS = np.arange(N-NBOUTPUTNEURONS, N-NBOUTPUTNEURONS + NBRESPNEURONS)
assert N-NBOUTPUTNEURONS+NBRESPNEURONS == N-NBAUXNEURONS  # The last neurons are output neurons, and the last output neurons are aux neurons.
AUXNEURONS = np.arange(N-NBAUXNEURONS, N)   # Again, no auxiliary neurons in this code.
REWARDNEURONS = np.arange(NBSTIMNEURONS, NBSTIMNEURONS+NBREWARDNEURONS) # The neuron receiving (and broadcasting) the "success" signal is the one just after the stimulus inputs
NOISENEURONS = np.arange(NBSTIMNEURONS+NBREWARDNEURONS, NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS)
BIASNEURONS = np.arange(NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS, NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS+NBBIASNEURONS)
FIRSTRESPSIGNALNEURON = NBSTIMNEURONS+NBREWARDNEURONS+NBNOISENEURONS+NBBIASNEURONS   # The first neuron that receives the response signal. We'll need this later
assert FIRSTRESPSIGNALNEURON == NBINPUTNEURONS
RESPSIGNALNEURONS = np.arange(FIRSTRESPSIGNALNEURON, FIRSTRESPSIGNALNEURON +NBRESPSIGNALNEURONS)

BIASVALUE = 1.0



NBTASKSPERGEN = 2 #  2 task blocks per generation


NBTRIALSLOSS = 100              # Evolutionary loss is evaluated over the last 100 trials of each block
NBTRIALS =  300 + NBTRIALSLOSS  # Total number of trials per block
# NBTRIALS =  4 + NBTRIALSLOSS 
NBNOISETRIALS = 0


WDECAY =  3e-4 # Evolutionary weight decay parameter (for the Adam optimizer)
L1PEN =  0 # No L1 penalty for now.
L1PENALPHA = 0 # Same for the alpha parameters
MUTATIONSIZE = .01 #  Std dev of the Gaussian mutations of the evolutionary algorithm

REWARDSIZE = 3.0 # Size of the binary reward signal (correct/incorrect)
STIMSIZE = 3.0 # Size of the stimulus input
RESPSIGNALSIZE = 3.0 # Size of the response signal
NOISESIZE = 0 # .2  # .1 # 0.1


# Generate all the inputs and targets of each trial, for all batch individuals, for a whole task block. That is, all the inputs and 
# targets of all trials are generated together for each block. We do that to ensure proper balancing.
def generateInputsAndTargetsForTask(mytask='watchstim1', whichresp=1):  # whichresp always 1 in this code, since NBRESPS = 1
    assert NBOUTPUTNEURONS == 1 + NBRESPS + NBAUXNEURONS
    assert whichresp in np.arange(1, NBRESPS+1)
    assert NBSTIMNEURONS == 2 
    assert mytask in alltasks
    assert NBTRIALS > NBTRIALSLOSS
    assert STIMTIME % 2 == 0


    returns=[]
    
    # We could just use random stimuli, but we want to balance the inputs and outputs.

    trials = []
    for trialpart in range(2): # We separately balance the trials that don't count for loss, and those that do. Then we concatenate at the end.
        nbt = NBTRIALSLOSS if  trialpart == 1 else (NBTRIALS - NBTRIALSLOSS)

        # Half inputs are 1, half are 0. Also, half identical b/w both stimuli, half different.
        stims1 = np.zeros((BS, nbt)).astype(int)
        stims2 = np.zeros((BS, nbt)).astype(int)
        stims1[:,:nbt//2] = 1
        stims2[:,::2] = 1

        # We need to shuffle independently in each row (i.e. for each batch element), but the same way for both stims1 and stims2 in each row
        # Taken from StackOverflow:
        p = np.argsort(np.random.rand(BS,nbt), axis=1)
        stims1 = (np.array(list(map(lambda x, y: y[x], p, stims1))))
        stims2 = (np.array(list(map(lambda x, y: y[x], p, stims2))))

        trials.append((stims1, stims2))

    # Now 'trials' is a list of 2 elements (non-loss trials and loss trials), each of which is a list of 2 elements (stims1, stims2), 
    # each of which is a long list of binary values (value of this particular stimulus for this particular trial).

    # We rearrange them into two numpy arrays: one for stims1 and one for stims2 (each including all trials, non-loss and loss)
    stims1 = np.concatenate((trials[0][0], trials[1][0]), axis=1)
    stims2 = np.concatenate((trials[0][1], trials[1][1]), axis=1)
    

    # Now we have two proper lists of binary value pairs, each giving the value of one stimulus input for each trial. Let's turn this 
    # into actual numerical inputs for the network:
    for numtrial in range(NBTRIALS):
        
        inpts = np.zeros((BS, NBSTIMNEURONS, STIMTIME)) # T)) # 
        
        StimDur = STIMTIME

        StartStim = 0

        # We're assuming that the first two neurons are stimulus neurons

        # The two stimuli are presented in succession, with both input neurons locked in opposite values to each other:
        inpts[:, 0, StartStim:StartStim+StimDur//2 - 2] = 2.0 * stims1[:, numtrial][:, None] - 1.0
        inpts[:, 0, StartStim+StimDur//2:StartStim+StimDur - 2] = 2.0 * stims2[:, numtrial][:, None] - 1.0
        inpts[:, 1, StartStim:StartStim+StimDur] = -inpts[:, 0, StartStim:StartStim+StimDur] 

    
        # Now we compute the targets, that is, the expected values of the output neurons, depending on inputs and tasks

        # The response is binary: either you respond (i.e. only the non-null response should be active), or you don't (only the null-response neuron should be active)

        # Note: For now, whichresp = NBRESPS = 1 - there is only one non-null response neuron (in addition to the null-response neuron).

        assert 1+whichresp >= 2; assert 1+whichresp < 2 + NBRESPS
        
        inpts = torch.from_numpy(inpts).float().to(device)


        # Generate the targets
        tgts = np.zeros((BS, NBRESPNEURONS, RESPONSETIME))

        # First we generate the expected output for the non-null response neuron, based on inputs and task:
        if mytask == 'watchstim1':
            tgts[:, whichresp, :] = stims1[:, numtrial][:, None]
        elif mytask == 'watchstim2':
            tgts[:, whichresp, :] = stims2[:, numtrial][:, None]
        elif mytask == 'antiwatchstim1':
            tgts[:, whichresp, :] = 1.0 - stims1[:, numtrial][:, None]
        elif mytask == 'antiwatchstim2':
            tgts[:, whichresp, :] = 1.0 - stims2[:, numtrial][:, None]
        elif mytask == 'and':
            tgts[:, whichresp, :] = (stims1[:, numtrial][:, None] * stims2[:, numtrial][:, None])
        elif mytask == 'nand':
            tgts[:, whichresp, :] = 1.0 - (stims1[:, numtrial][:, None] * stims2[:, numtrial][:, None])
        elif mytask == 'or':
            tgts[:, whichresp, :] = np.clip(stims1[:, numtrial][:, None] + stims2[:, numtrial][:, None], 0.0, 1.0)
        elif mytask == 'nor':
            tgts[:, whichresp, :] = 1.0 - np.clip(stims1[:, numtrial][:, None] + stims2[:, numtrial][:, None], 0.0, 1.0)
        elif mytask == '10':
            tgts[:, whichresp, :] = stims1[:, numtrial][:, None] * (1.0 - stims2[:, numtrial][:, None])
        elif mytask == 'anti10':
            tgts[:, whichresp, :] = stims1[:, numtrial][:, None] * (1.0 - stims2[:, numtrial][:, None])
        elif mytask == '01':
            tgts[:, whichresp, :] = (1.0 - stims1[:, numtrial][:, None]) * stims2[:, numtrial][:, None]
        elif mytask == 'anti01':
            tgts[:, whichresp, :] = 1.0 - (1.0 - stims1[:, numtrial][:, None]) * stims2[:, numtrial][:, None]
        elif mytask == 'dms':
            tgts[:, whichresp, :] = (stims1[:, numtrial][:, None] == stims2[:, numtrial][:, None])
        elif mytask == 'dnms':
            tgts[:, whichresp, :] = (stims1[:, numtrial][:, None] != stims2[:, numtrial][:, None])
        else:
            raise ValueError("Task not in list of allowed tasks! Task is"+str(mytask))

        assert np.all(np.logical_or(tgts == 0.0 , tgts == 1.0))

        # The null-response neuron's expected output is just the opposite of the non-null response neuron output (either you respond, or you don't !)
        tgts[:, 0, :] = 1.0 - tgts[:, whichresp, :]

        tgts = torch.from_numpy(tgts).float().to(device)     

        # In practice, we clip targets to 0.1/0.9 instead of actually 0.0/1.0. This may or may not help.
        tgts.clip_(min=0.1, max=0.9)



        # Finally: antithetic pairs get  identical sequences of trials !
        inpts[BS//2:, :, :] = inpts[:BS//2, :, :]
        tgts[BS//2:, :, :] = tgts[:BS//2, :, :]




        returns.append((inpts, tgts))
    
    return returns


totalnbtasks = 0
ticstart = time.time()
with torch.no_grad():

    # traincurves=[]
    evolosses_allruns=[]
    evolossesaux_allruns=[]
    evolossesconc_allruns=[]
    binarylosses_allruns=[]

    ml0sallruns = []
    bl0sallruns = []
    
    PRINTING = True # if numgen == 0 or np.random.rand() < .05 else False


    # The code allows to have multiple runs in succession, all totally independent from each other. 
    # In practice, this is useful if you don't have too many generations per run (as with the NAND test task). 
    # For the DMS test task, we only use one run because it requires so many generations that multiple runs in the same session 
    
    for numrun in range(NBRUNS):

        # Initialize innate weights values
        w =  torch.randn(N,N)  * JINIT / np.sqrt(N) 
        w = w.to(device)
        
        # Initialize alpha values - the plasticity parameters (capital-pi in the paper)
        alpha = INITALPHA * torch.ones_like(w).to(device)

        # We zero out input weights to input neurons, though it probably doesn't have any effect.
        w.data[:NBINPUTNEURONS, :] = 0   # Each *row* of w contains the weights to a single neuron.
        winit = w.clone()

        # We will be using the Adam optimizer to apply our (hand-computed) evolutionary gradients
        optimizer = optim.Adam([w, alpha], lr=LR, weight_decay=WDECAY)  # Default betas=(0.9, 0.999)

        # Evolosses are real-valued losses used for evolution. Binarylosses are binary 'correct/wrong' signals, also used for logging.
        evolosses = []
        responses0 = []
        binarylosses = []
        wgradnorms = []
        mytaskprev = mytaskprevprev = mytaskprevprevprev = whichrespprev = whichrespprevprev = whichrespprevprevprev = -1
        

        # Ready to start the evolutionary loop, iterating over generations (i.e. lifetimes). 

        for numgen in range(NBGEN):

            # Every 10th generation is for testing on the withheld task (with no weight change)
            TESTING = False
            if numgen == 0 or numgen == NBGEN-1 or numgen % 10 == 0:
                TESTING = True
                if PRINTING:
                    print("TESTING")


            tic = time.time()   
            responses0thisgen = []
        
            # Generating the population of mutated individuals:

            # First, batch the weights.
            bw = torch.dstack(BS*[w]).movedim(2,0).to(device)     # batched weights
            balpha = torch.dstack(BS*[alpha]).movedim(2,0).to(device)     # batched alphas
            # Generate the mutations, for both w and alpha
            # NOTE: batch element 0 (and BS/2, its antithetic pair) are NOT mutated, represent the curent unmutated candidate genotype.
            mutations_wandalpha = []
            for  n, x in enumerate( (bw, balpha) ):
                mutations = torch.randn_like(x, requires_grad=False).to(device) *  MUTATIONSIZE
                mutations[0,:,:] = 0  # 1st item in batch = current candidate
                mutations[BS//2:, :, :] = -mutations[:BS//2, :, :]    # Antithetic sampling for mutations ! Really helps.
                if TESTING:
                    mutations *= 0.0  # Not strictly necessary (no weight change is applied) - results in batch score variance being caused only by randomness in trial order
                x += mutations  
                mutations_wandalpha.append(mutations)


            
            bw.data[:, :NBINPUTNEURONS, :] = 0  # Input neurons receive 0 connections. Probably not necessary.
            bworig = bw.clone()                 # Storing the weights for comparison purposes at the gradient step (below).

            responsecounts = torch.zeros(BS, NBRESPS + 1).to(device)
            lifelosses = torch.zeros(BS, requires_grad=False).to(device)
            lifemselosses = torch.zeros(BS, requires_grad=False).to(device)
            # lifemselossesaux = torch.zeros(BS, requires_grad=False).to(device)
            lifeblosses = torch.zeros(BS, requires_grad=False).to(device)
                
            allresps=[]


            # Lifetime loop, iterates over task-blocks:
            for numtask in range(NBTASKSPERGEN):
                totalnbtasks += 1

                # bpw = batched plastic weights
                bpw = torch.zeros_like(bw).to(device)  # For now, plastic weights are initialized to 0 at the beginning of each task.

                # Initialize neural states
                bstates = .1 * torch.ones(BS, N).to(device)  # bstates (batched states) contains the neural activations (before nonlinearity). Dimensionality appropriate for batched matrix multiplication. 
                bstates[:, INPUTNEURONS] = 0
                bresps = 1.0 * bstates  # bresps is the actual neural responses, after nonlinearity, and also serve as the input for the next step.
                bresps[:, BIASNEURONS] = BIASVALUE

                meanlosstrace = torch.zeros(BS, 2 * 2).to(device)
                bls = []    # Will store binary losses of all batch elements, for each trial of this task
                bl0s = []   # Same but only for batch element 0 (i.e. the unmutated candidate genome)
                ml0s = []   # MSE loss 


                # Choose the task ! If not testing, makes sure it's different from recently chosen tasks.

                if TESTING: 
                    mytask = TESTTASK 
                    mytasknum = alltasks.index(mytask)
                    whichresp = 1   # This is always 1 for now, because NBRESP=1, i.e. we only use one response output (other than the null, "no-response" output)
                else:
                    while True:
                        mytasknum = np.random.randint(len(alltasks))

                        mytask = alltasks[mytasknum]
                        whichresp = 1 

                        if ( (mytask!= TESTTASK)  
                            and (mytask != TESTTASKNEG)  # We withhold both the test task and its logical negation
                            and     (mytask != mytaskprev) 
                                and (mytask != mytaskprevprev) 
                        ):

                            break

                    mytaskprevprev = mytaskprev; mytaskprev= mytask
                    whichrespprevprev = whichrespprev; whichrespprev = whichresp
                
                # Cumulative MSE and binary losses for this task, over the last NBLOSSTRIALS of the block:
                taskmselosses = torch.zeros_like(lifemselosses).to(device)
                taskblosses = torch.zeros_like(lifemselosses).to(device)

                if PRINTING:
                    print("task", mytask, "whichresp:", whichresp)
                
                # OK, ready to start the task.

                # Generate the task data (inputs and targets) for all trials:
                taskdata = generateInputsAndTargetsForTask(mytask=mytask, whichresp=whichresp)

                eligtraces =   torch.zeros_like(bw, requires_grad=False).to(device)  # Initialize the eligibility traces at the start of each block/task.

                # Task loop, iterating over trials
                # You do NOT erase memory (neural activations or plastic weights) between successive trials ! 
                for numtrial in range(NBTRIALS):
                    
                    # Initializations
                    losses = torch.zeros(BS, requires_grad=False).to(device)     # Losses for this trial
                    mselossesthistrial = torch.zeros(BS, requires_grad=False).to(device)     # Losses for this trial
                    totalouts = torch.zeros(BS, NBOUTPUTNEURONS, requires_grad=False).to(device)  

                    # Extract the inputs and targets for this trial:
                    inputs, targets = taskdata[numtrial]
                    assert inputs.shape == (BS, NBSTIMNEURONS, STIMTIME)

                    # Run the network. Trial loop, iterating over timesteps
                    for numstep in range(T):          

                        # Update neural activations
                        bstates += (DT / TAU) * (-bstates +  torch.bmm((bw + balpha * bpw), bresps[:, :, None])[:,:,0] )  


                        # Applying the random perturbations on neural activations, a.k.a. "modulations", both for noise and for the lifetime plasticity algorithm (node-perturbation)
                        # Ans also updating the eligibility trace appropriately
                        if numstep > 1 : 
                            modulindices =  (torch.rand(1, N) < PROBAMODUL).int()   # Which neurons get perturbed?
                            modulations = (ALPHAMODUL * modulindices * (2 * torch.rand(1, N) - 1.0)).to(device)  # Note the dimensions: the same noise vector is applied to all elements in the batch (to save time!)
                           
                            bstates += modulations
                            
                            # Node-perturbation: Hebbian eligibility trace = product between inputs (bresps from previous time step) and *perturbations* in outputs. dH = X * deltaY 
                            # We do this with a (batched) outer product between the (column) vector of modulations (1 per neuron) and the (row) vector of inputs
                            # Note that here, since we have an RNN, the input is bresps - the network's responses from the previous time step
                            if torch.sum(modulindices) > 0:
                                eligtraces += torch.bmm( modulations.expand(BS, -1)[:, :, None],  bresps[:, None, :] ) 

                        # Eligibility traces, unlike actual plastic weights, are decaying
                        eligtraces -=  (DT / TAU_ET) * eligtraces


                        # We can now compute the actual neural responses for this time step, applying the appropriate nonlinearity to each neuron
                        bresps = bstates.clone() # F.leaky_relu(bstates)
                        # The following assumes that output neurons are the last neurons of the network !                        
                        bresps[:,N-NBOUTPUTNEURONS:].sigmoid_()     # The output neurons are sigmoids, all others are tanh. An arbitrary design choice.
                        bresps[:,:N-NBOUTPUTNEURONS].tanh_()
                        

                        torch.clip_(bresps, min=-5, max=5)  # Unnecessary if all nonlinearities are bounded.


                        # Are we in the input presentation period? Then apply the inputs.
                        # Inputs are clamping, fixing the response of the input neurons.
                        if numstep < STIMTIME:
                            bresps[:, STIMNEURONS] = STIMSIZE * inputs[:, :, numstep]        
                        else:
                            bresps[:, STIMNEURONS] = 0

                        bresps[:, BIASNEURONS] = BIASVALUE


                        # Are we in the response period? Then collect network response.
                        if numstep >= STARTRESPONSETIME and numstep < ENDRESPONSETIME:

                            assert numstep < STARTREWARDTIME
                            # Accumulate the total activation of each output neuron, so that we can compute the network's actual response at the end of response period:
                            totalouts +=  bresps[:, OUTPUTNEURONS] 
                            # Accumulate the MSE error between actual and expected outputs:
                            mselossesthistrial += torch.sum( (bresps[:, RESPNEURONS] - targets[:, :, numstep - STARTRESPONSETIME]) ** 2, axis=1 ) / RESPONSETIME

                        else:
                            bresps[:, OUTPUTNEURONS] = 0.0


                        # Is the response period finished, or equivalently, are we at the first step of the reward / feedback period?
                        # If so, compute the network's response and apply plasticity (i.e. deliver the neuromodulation).
                        if numstep == STARTREWARDTIME:
                            # The network's response for this trial (0 or 1) is the index of the output neuron that had the highest cumulative output over the response period
                            totalresps = totalouts[:, :NBRESPNEURONS]   # Assumes that the response neurons are the first among output neurons !
                            responses = torch.argmax(totalresps, dim=1)  # responses is a 1D, integer-valued array of size BS. totalresps is a 2D real-vlued array of size BS, NBRESPS+1                           
                            
                            # blosses (binary losses) is a 1/-1 "correct/wrong" signal for each batch element for this trial.
                            blosses = 2.0 * (responses == torch.argmax(targets[:, :, 0], dim=1)).float() - 1.0    
                            responses0thisgen.append(float(responses[0]))

                            # We also want the 1-hot version of the response for each neuron. This will be used as the response signal below.
                            if numtrial > 0:
                                responses1hot_prev = responses1hot.clone()
                            responses1hot = F.one_hot(responses, 1+NBRESPS)
                            if numtrial < NBTRIALS - NBTRIALSLOSS:
                                responsecounts += responses1hot

                            # Now we apply lifetime plasticity, with node-perturbation, based on eligibility trace and suitably baselined reward/loss

                            # We compute separate baseline (running average) losses for different types of trials, as defined by their inputs (as in Miconi, eLife 2017). 
                            # So we need to find out the trial type for each element in batch.
                            input1 = inputs[:, 0, 0]; input2 = inputs[:, 1, 0]  # This assumes that inputs are constant and start from timestep 0 !
                            trialtypes = (input1>0).long() * 2 + (input2>0).long()

                            # Compute and apply the plasticity, based on accumulated eligibility traces and reward                          
                            if numtrial > 30:  # Lifetime plasticity is only applied after a few burn-in trials.
                                # eligtraces: BS x N x N.   mselossesthhistrial:  BS.    meanlosstrace: BS x (N.N).    trialtypes: BS
                                # dw should have shape BS x N x N, i.e. one for each connection and batch element. Do not sum over batch dimension! The batch is purely evolutionary !


                                dw =   (ETA * eligtraces  * (  meanlosstrace[np.arange(BS), trialtypes] * (mselossesthistrial - meanlosstrace[np.arange(BS), trialtypes]) )[:, None, None]).clamp(-MAXDW, MAXDW)
                                # dw =   (ETA * eligtraces  * (  meanlosstrace[np.arange(BS), trialtypes] * (mselossesthistrial ) )[:, None, None]).clamp(-MAXDW, MAXDW)


                                bpw -= dw

                                if PRINTING and np.random.rand() < .02:
                                    print(numrun, numtrial, "{:.4f}".format(float(torch.mean(mselossesthistrial))), "{:.4f}".format(float(torch.norm(bw+bpw))), 
                                                                        "{:.4f}".format(float(torch.norm(dw))), "{:.4f}".format(float(torch.norm(bpw))))
                            # Updating the baseline - running average of losses, for each batch element, for the trial type just seen
                            meanlosstrace[torch.arange(BS).long(), trialtypes] *= MULOSSTRACE
                            meanlosstrace[torch.arange(BS).long(), trialtypes] +=  (1.0 - MULOSSTRACE) * mselossesthistrial



                        # Are we in the reward signal period?
                        # The neuromodulatory reward signal is applied just once per trial, above. Here we provide a binary "correct/ incorrect" signal to the network, 
                        # i.e. "was my response right or wrong for this trial?" 
                        # We also provide a signal indicating which response it gave in this trial (in theory it should be able to calculate it itself if needed, but this may help)
                        if numstep >= STARTREWARDTIME and numstep < ENDREWARDTIME: # Note that by this time, the loss has been computed and is fixed
                            
                            # We provide a binary, "correct/incorrect" signal to the network
                            bresps[:,REWARDNEURONS[0]] = REWARDSIZE * blosses[:]         # Reward input is also clamping
                            bresps[:,REWARDNEURONS[1]] = -REWARDSIZE * blosses[:]         # Reward input is also clamping

                            bresps[:,REWARDNEURONS].clip_(min=0)

                            # We provide the network with a signal indicating the actual response it chose for this trial.
                            # bresps[:, RESPSIGNALNEURONS] = responses1hot.float()
                            # # The following is probably unnecessarily obscure and may be replaced with simply putting in the one-hot version of responses multiplied by Rewardsize:
                            bresps[:, RESPSIGNALNEURONS] = 0
                            bresps.scatter_(1, FIRSTRESPSIGNALNEURON + responses[:, None],   RESPSIGNALSIZE)  # The ResponseSignalNeuron corresponding to the actual response gets value RESPSIGNALSIZE - all others get 0

                            
                        else:
                            bresps[:,REWARDNEURONS] = 0
                            bresps[:, RESPSIGNALNEURONS] = 0


                   
                    # Now all steps done for this trial:
                
                    if PRINTING:
                        if np.random.rand() < .1: 
                            print("|", int(responses[0]), int(blosses[0]), end=' ')
                    
                    ml0s.append(float(mselossesthistrial[0]))
                    bl0s.append(float(blosses[0]))
                    bls.append(blosses.cpu().numpy())
 

                    # If this trial is part of the last NBTRIALSLOSS, we accumulate its trial loss into the agent's total loss for this task.
                    if numtrial >= NBTRIALS - NBTRIALSLOSS:     # Lifetime  losses are only estimated over the last few trials
                        taskmselosses += 2 * mselossesthistrial / NBTRIALSLOSS   # the 2* doesn't mean anything
                        taskblosses += blosses / NBTRIALSLOSS
                

                # Now all trials done for this task:
                if PRINTING:
                    print("Med task mseloss:", "{:.4f}".format(float(torch.median(taskmselosses))))


                    # print("")
                lifemselosses += taskmselosses / NBTASKSPERGEN 
                lifeblosses += taskblosses / NBTASKSPERGEN 
            
                if (TESTING or numgen == 0) and numtask == 0:
                    # These files contain respectively the first and *latest* Testing block of the *current* run only. 
                    FNAME = 'bl1block_gen0.txt' if numgen == 0 else 'bl1block_lastgen.txt'
                    # np.savetxt(FNAME, np.array(bl0s))
                    np.savetxt(FNAME, np.vstack(bls))
                if numtask == 0 and (numgen == 0 or numgen == NBGEN-1):
                    # We append the 1st and last block of each run, all together alternating in the same array
                    assert (TESTING)
                    ml0sallruns.append(ml0s)
                    bl0sallruns.append(bl0s)


            # After all tasks done for this lifetime / generation:

            # if (numgen+1) == NBGEN // 3 or (numgen+1) == 2 * NBGEN // 3:
            #     for g in optimizer.param_groups:
            #         # g['lr'] /= 4.0 
            #         g['lr'] /= 10.0 
            #         g['weight_decay'] /= 10.0


            # responseprobas = responsecounts /  torch.sum(responsecounts, dim=1, keepdim=True)
            # responseprobassq = responseprobas * responseprobas
            # concentrations = torch.sum(responseprobassq, dim=1)

            lifelosses = lifemselosses 

            binarylosses.append(float(lifeblosses[0]))
            evolosses.append(float(lifemselosses[0]))

            np.savetxt('blosses_onerun.txt', np.array(binarylosses))
            np.savetxt('mselosses_onerun.txt', np.array(evolosses))
            

            # Now we're ready to perform evolution (by computing gradients by hand, and then applying the optimizer with these gradients)
            optimizer.zero_grad()

            # Gradient is just loss x mutation (remember we use antithetic sampling)
            gradient = torch.sum(mutations_wandalpha[0] * lifelosses[:, None, None], axis=0) # / BS
            wgradnorm = float(torch.norm(gradient))
            wgradnorms.append(wgradnorm)
            if PRINTING:
                print("norm w:", "{:.4f}".format(float(torch.norm(w))), "norm gradient:", "{:.4f}".format(wgradnorm), 
                        "norm a:", "{:.4f}".format(float(torch.norm(alpha))), "mean a:",  "{:.4f}".format(float(torch.mean(alpha))))


            w.grad = gradient
            wprev = w.clone()

            gradientalpha = torch.sum(mutations_wandalpha[1] * lifelosses[:, None, None], axis=0) # / BS
            alpha.grad = gradientalpha
            alphaprev = alpha.clone()

            if numgen > 0 and not TESTING:
                optimizer.step()
                w -= L1PEN * torch.sign(w)
                alpha -= L1PENALPHA * torch.sign(alpha)
            
            wdiff = w - wprev
            adiff = alpha - alphaprev
            if PRINTING:
                print("Norm w-wprev:", "{:.4f}".format(float(torch.norm(wdiff))), "Max abs w-wprev:", "{:.4f}".format(float(torch.max(torch.abs(wdiff)))), 
                    "Norm a-aprev:", "{:.4f}".format(float(torch.norm(adiff))), "Max abs a-aprev:", "{:.4f}".format(float(torch.max(torch.abs(adiff))))  )

        

            if PRINTING:
                print("Med/min/max/Half-Nth/0th loss in batch:", float(torch.median(lifelosses)), float(torch.min(lifelosses)), float(torch.max(lifelosses)),
                                        float(lifelosses[BS//2]), float(lifelosses[0]))
                print("Gen", numgen, "of run", numrun, "done in", time.time()-tic)
        


        # After all generations for this run are done:

        # These arrays contain the first and last block (both on test task) of each run to date, in alternation
        mlnp = np.vstack(ml0sallruns)
        np.savetxt('meanlosses0_1stLastB_allruns.txt', mlnp)
        blnp = np.vstack(bl0sallruns)
        np.savetxt('blosses0_1stLastB_allruns.txt', blnp)

        binarylosses_allruns.append(binarylosses)
        evolosses_allruns.append(evolosses)

        with open('binlossDMS_LR'+str(LR)+'MutSize'+str(MUTATIONSIZE)+'NoiseS'+str(NOISESIZE)+'_RSize'+str(REWARDSIZE)+'SSize'+str(STIMSIZE)+'.txt', 'w') as myfile:
            for c in binarylosses_allruns:
                for l in c:
                        myfile.write(str(l)+" ")
                myfile.write("\n")
        with open('evolossDMS_LR'+str(LR)+'MutSize'+str(MUTATIONSIZE)+'NoiseS'+str(NOISESIZE)+'_RSize'+str(REWARDSIZE)+'SSize'+str(STIMSIZE)+'.txt', 'w') as myfile:
            for c in evolosses_allruns:
                for l in c:
                        myfile.write(str(l)+" ")
                myfile.write("\n")



print("Time taken:", time.time()-ticstart)

In [ ]:
import matplotlib.pyplot as plt; import numpy as np

# Showing the success rate over the batch (i.e. the mean binary loss over the batch) for each trial of the last test-task block.

z = .5 + .5 * np.vstack(bls)
print(z.shape)
plt.plot(np.mean(z, axis=1))

In [ ]:
# % correct over last 100 trials of each generation (unmutated candidate genotype only), shown separately for the non-test tasks (blue) and test task (red)
# Median and IQR over all runs. 

bl = .5 + .5 * np.loadtxt('binlossDMS_LR0.003MutSize0.01NoiseS0_RSize3.0SSize3.0.txt')
if(len(bl.shape)<2):   # If there is only a single run, add a singleton dimension
    bl = bl[None, :]
print(bl.shape)
ss = bl.shape[1]  # Number of generations

plt.figure(figsize=(4,4))

xr = np.arange(len(bl[0,:]))
plt.fill_between(xr[xr%10 != 0], np.quantile(bl, .25, axis=0).T[xr % 10 != 0], np.quantile(bl, .75, axis=0).T[xr % 10 != 0], color='b', alpha=.3)
plt.plot(xr[xr % 10 != 0], np.quantile(bl, .5, axis=0).T[xr % 10 != 0], 'b', label='Training tasks');
plt.fill_between(xr[::10], np.quantile(bl, .25, axis=0).T[0::10], np.quantile(bl, .75, axis=0).T[0::10], color='r', alpha=.3)
plt.plot(xr[::10], np.quantile(bl, .5, axis=0).T[0::10], 'r', label='Test task')


plt.xlabel('Generations')
plt.ylabel('% correct over last 100 trials')
plt.legend(loc='lower right')

plt.title('Test task: '+str(TESTTASK).upper())

plt.tight_layout()

plt.savefig('evoloss'+str(TESTTASK).upper()+'.png', dpi=300)


In [ ]:
# % correct over all *runs* (unmutated candidate genotype only), for the first and last block (both on the test task).

from matplotlib.ticker import FormatStrFormatter

bla = np.loadtxt('blosses0_1stLastB_allruns.txt')
print(bla.shape)
nbruns = bla.shape[0]//2

plt.subplot(2,1,1);
mm = np.mean(bla[0::2, :]>0, axis=0)
plt.plot(mm, 'g')
plt.plot(np.arange(mm.size), .5*np.ones_like(mm), 'k:')
plt.ylim([-.1, 1.1])
#mm = np.mean(bla[1::2, ::2]>0, axis=0)
#plt.plot(np.arange(0,400,2), mm, 'g')
#plt.plot(smooth(mm), 'k')
plt.ylabel("% Correct (over "+str(nbruns)+" runs)")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.twinx()
plt.gca().get_yaxis().set_ticks([])
plt.ylabel("Generation 0")

plt.subplot(2,1,2);
mm = np.mean(bla[1::2, :]>0, axis=0)
plt.plot(mm, 'g')
plt.plot(np.arange(mm.size), .5*np.ones_like(mm), 'k:')
plt.ylim([-.1, 1.1])
#mm = np.mean(bla[1::2, ::2]>0, axis=0)
#plt.plot(np.arange(0,400,2), mm, 'g')
#plt.plot(smooth(mm), 'k')
plt.ylabel("% Correct (over "+str(nbruns)+" runs)")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.xlabel("Trials")
plt.twinx()
plt.gca().get_yaxis().set_ticks([])
plt.ylabel("Generation "+str(ss))

plt.tight_layout()
plt.savefig('trials'+str(TESTTASK).upper()+'.png', dpi=300)

In [ ]:
print("Med/min/max/Half-Nth/0th loss in batch:", float(torch.median(lifelosses)), float(torch.min(lifelosses)), float(torch.max(lifelosses)),
      float(lifelosses[BS//2]), float(lifelosses[0]))


In [ ]:
print(inputs.shape, targets.shape)

print(mytask)
for nn in range(6):
    print("Trial", nn)
    ii, tt = taskdata[nn]
    print(ii[0,0, 0::15])  # Stim Input 0 for this trial, timesteps 0 and 15, batch element 0
    print(tt[0, :, 0])  # Targets for this trial, timestep 0,  batch element 0

for nn in range(6):
    print("Trial", nn)
    ii, tt = taskdata[nn]
    print(ii[1,0, 0::15])  # Stim Input 0 for this trial, timesteps 0 and 15, batch element 0
    print(tt[1, :, 0])  # Targets for this trial, timestep 0,  batch element 0


In [ ]:
# This was code for showing the trajectories of multiple neurons over several trials, but we don't store them ('allresps') any more due to mempry constraints

if False:
    ar1 = torch.dstack(allresps) # Now allresps has dimensions batchsize x neurons x time

    # If the network is not chaotic enough and they all have the same weights, they will produce very similar final outputs independently of initial conditions - because eigenvectors !
    ar = ar1.cpu().numpy()
    print(ar.shape)
    ar[ar>1.5] = 1.5
    ar[ar<-1.5] = -1.5

    plt.figure(figsize=(20,20))
    z = plt.plot(ar[1,:20,-1000:].T)  # 20 first neurons of batch element 1
    z = plt.plot(ar[1,-1,-1000:].T, 'r--')  # Last (response) neuron of batch element 1
    z = plt.plot(ar[1,-2,-1000:].T, 'b--')  # Last (response) neuron of batch element 1
    plt.figure(figsize=(10,10))
    z = plt.plot(ar[0,:20,-1000:].T)  # 20 first neurons of batch element 0
    z = plt.plot(ar[0,-1,-1000:].T, 'r--')  # Last (response) neuron of batch element 0
    z = plt.plot(ar[0,-2,-1000:].T, 'b--')  # Last (response) neuron of batch element 0


In [ ]:
# Show the final evolved w matrix (innate weights), and its column and row averages

import matplotlib.pyplot as plt; import numpy as np

ww = w.cpu().numpy()
print("Std W:", np.std(ww), "Norm W", np.sqrt(np.sum(ww**2)))
zz = np.random.randn(N,N)  * JINIT / np.sqrt(N) 
print("Std w_init_like:", np.std(zz), "norm:", np.sqrt(np.sum(zz**2)))
print(np.min(ww), np.max(ww), np.median(ww))
print(np.min(zz), np.max(zz), np.median(zz))

plt.matshow(ww)
plt.figure()
plt.plot(np.mean(ww, axis=0), 'r')
plt.figure()
plt.plot(np.mean(ww, axis=1), 'b')

In [ ]:
# Show the final evolved alpha matrix (plasticity parameters, capital-Pi in the MML paper)


aa = alpha.cpu().numpy()

print(np.mean(aa))
print(np.min(aa), np.max(aa), np.median(aa))
plt.matshow(aa)
plt.figure()
plt.plot(np.mean(aa, axis=0), 'r')
plt.figure()
plt.plot(np.mean(aa, axis=1), 'b')

In [ ]:
# Show the mean plastic weights over the whole batch., and for two elements in the batch (note that this is after the end of the last block)

pw = bpw.cpu().numpy()

print(pw.shape)
plt.matshow(np.mean(pw, axis=0))
print(np.mean(np.abs(w.cpu().numpy())), np.mean(np.abs(pw)))
print(np.max(np.abs(w.cpu().numpy())), np.max(np.abs(pw)))
print(np.min(pw[0,:,:]), np.max(pw[0,:,:]), np.median(pw[0,:,:]))
plt.figure()
plt.matshow(pw[0,:,:])
plt.figure()
plt.matshow(pw[1,:,:])


In [ ]:
ww = w.cpu().numpy()
pw0 = bpw[0,:,:].cpu().numpy()
aa = alpha.cpu().numpy()
np.savetxt('w.txt', ww)
np.savetxt('pw0.txt', pw0)
np.savetxt('alpha.txt', aa)